## Task. Customer Segmentation

- 데이터를 정제해서, 고객별로 재정렬이 필요합니다. (aggregation)


- row가 고객별 데이터로 묶이고 난 다음, 고객들을 유형별로 나눠봅니다.


- 여러 가지 클러스터링 알고리즘을 사용하여, 결과를 테스트해봅니다.


- 클러스터링을 위한 전처리부터, 평가까지 모든 항목을 하나하나 살펴보면서 데이터를 뜯어봅니다.

### Data Description

Source : https://www.kaggle.com/c/instacart-market-basket-analysis

### 1. 데이터 불러오기 

In [ ]:
data_path = "../input/instacart-market-basket-analysis/"

In [ ]:
start, end = [int(x) for x in input("돌려보고 싶은 클러스터 개수의 시작과 끝 범위를 입력해주세요 : ").split(",")]
cluster_model = input("사용할 클러스터링 모델을 입력하세요(kmeans/hac/dbscan/spectral) : ")
column_level = input("user matrix에 사용할 column을 입력하세요(department/aisle/product_name) : ")
PCA_mode = True
if PCA_mode:
    n_components = int(input("PCA에 사용할 n_components 개수를 입력하세요 : "))
    
quick_test = False
K = list(range(start, end+1))

## 1. Data Preparation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

orders = pd.read_csv(data_path + "orders.csv")
if quick_test: prior = pd.read_csv(data_path + "order_products__prior.csv")[:10000]
else : prior = pd.read_csv(data_path + "order_products__prior.csv")
train = pd.read_csv(data_path + "order_products__train.csv")
products = pd.read_csv(data_path + "products.csv")
aisle = pd.read_csv(data_path + "aisles.csv")
department = pd.read_csv(data_path + "departments.csv")

In [ ]:
# 불러온 모든 테이블을 합칩니다.
temp = pd.merge(prior, products, on=["product_id"])
temp = pd.merge(temp, orders, on=["order_id"])
temp = pd.merge(temp, aisle, on=["aisle_id"])
data = pd.merge(temp, department, on=["department_id"])
del temp
data

In [ ]:
# 거래내역에 대해서 각 물품을 얼마나 샀을까?
if quick_test:
    display(data.product_name.value_counts()[:10]) # top10

In [ ]:
# 고객의 산 물건의 수
if quick_test:
    display(data.user_id.value_counts()[:10])

In [ ]:
# 거래내역에 포함된 소분류별 개수
if quick_test:
    display(data.aisle.value_counts()[:10])

In [ ]:
# 거래내역에 포함된 대분류별 개수
if quick_test:
    display(data.department.value_counts()[:10])

### 2. 데이터 전처리

- 데이터를 transaction 단위로 변경합니다.

- 각자의 방법대로 데이터의 단위를 정해봅시다.

- 결측치를 처리하고, 정규화도 진행해봅니다.

- 필요하면 PCA나 SVD를 사용해도 상관없습니다.


> User 단위로 어떤 물품을 구매했는지의 정보만 가지는 feature vector로 변환한다. e.g. pd.crosstab, CountVectorizer

In [ ]:
columns = ["product_name", "user_id", "aisle", "department"] #data.columns
columns = np.array(columns)
columns = np.setdiff1d(data.columns, columns) # 차집합 구하는 함수.
data.drop(columns=columns, inplace=True)

In [ ]:
if column_level == "department": user_matrix = pd.crosstab(data.user_id, data.department) # 21d
elif column_level == "aisle" :  user_matrix = pd.crosstab(data.user_id, data.aisle) # 134d
else: user_matrix = pd.crosstab(data.user_id, data.product_name) #csr_matrix ## sparse matrix
user_matrix

### 3. 클러스터링 모델 적용하기

- 사용하는 클러스터링 모델은 KMeans와 AgglomerativeClustering으로 합니다.

(원하시면 DBSCAN이나 SpectralClustering을 사용해보셔도 됩니다. 단, 시간이 매우매우 오래 걸릴수 있으니 주의하세요..)

**[K-Means]**

- Elbow method를 이용하여 최적의 K값을 찾아보세요.


- sparse한 특징을 가지는 데이터를 클러스터링 하기 위해서는 어떤 기법을 사용해야 할까요?


- 클러스터링 결과를 시각화해보고, 실루엣 지수도 계산해봅시다.



**[Hierarchical Clustering]**

- 클러스터 개수를 4로 지정하고, linkage와 affinity를 바꿔가면서 실험해보세요.


- 어떤 linkage와 affinity를 쓸지 고민하려면, 어떤 방법을 사용해보면 좋을까요?


- 클러스터링 결과를 시각화해보고, 실루엣 지수도 계산해봅시다.

In [ ]:
X = user_matrix.values
print(X.shape)

if quick_test:
    from sklearn.manifold import TSNE

    # tSNE : 시각화용도의 차원감소 기법. (2차원으로 변환해주는 기법)
    tsne = TSNE(n_components=2)
    #tsne.fit()
    #tsne.transform()
    reduced_data = tsne.fit_transform(X)
    reduced_data

In [ ]:
if quick_test:
    # 206209 x 2
    plt.figure(figsize=(12, 12))
    #sns.scatterplot(data=reduced_data)
    plt.scatter(reduced_data[:, 0], reduced_data[:, 1], s=5, alpha=0.3)
    plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
#from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from tqdm import tqdm_notebook

def find_optimal_clusters(data, K):


    scores = [] # initialization

    for n_cluster in tqdm_notebook(K):
        #model = KMeans(n_clusters=n_cluster) # 2~10
        model = MiniBatchKMeans(n_clusters=n_cluster, batch_size=1024)
        pred = model.fit_predict(data)

        score = silhouette_score(data, pred)
        scores.append(score)
        
    optimal_K = np.array(scores).argmax() + K[0] # K
    best_pred = KMeans(n_clusters=optimal_K).fit_predict(data)
    if quick_test:
        return best_pred, scores
    else:
        return best_pred

In [ ]:
# Find optimal K
# 1) elbow method  -> yellowbrick    # 설치 이슈.
# from yellowbrick.cluster import elbow

#     elbow()

# 2) Silhouette score   # sklearn
if quick_test:
    best_pred, scores = find_optimal_clusters(X, K)
else:
    best_pred = find_optimal_clusters(X, K)
print("Find optimal K.")

In [ ]:
# 클러스터 개수별 실루엣 지수를 그려주는 그래프.
if quick_test:
    plt.figure(figsize=(8, 4))
    plt.title("Silhouette Score in range %d-%d" % (K[0], K[-1]), fontsize=14)
    plt.xlabel("Number of Clusters")
    plt.ylabel("Silhouette Score")
    plt.plot(K, scores)
    plt.show()

In [ ]:
if quick_test:
    # 클러스터별 색칠 공부
    plt.figure(figsize=(8, 8))
    plt.scatter(reduced_data[:, 0], reduced_data[:, 1], s=10, alpha=0.3, c=best_pred)
    plt.show()

In [ ]:
if quick_test:
    # 실루엣 계산
    print("Silhouette score : %.4f" % silhouette_score(X, best_pred)) # [-1, 1]

In [ ]:
# 차원이 큰 경우엔?
# 차원의 저주 문제를 해결하기 위해서 차원 감소 기법인 PCA를 적용해봅니다.
if PCA_mode:
    from sklearn.decomposition import PCA

    # tsne와 같습니다.
    # 1) n_components가 int면, 해당 차원으로 감소. # 2) n_components가 float면 해당 비율만큼 보존하는 차원으로 감소.
    pca = PCA(n_components=n_components)
    reduced_pca = pca.fit_transform(X)
    print(reduced_pca.shape)
    
    pca_columns = [f"PC_{n}" for n in range(1, n_components+1)]

    pca_df = pd.DataFrame(data=reduced_pca, columns=pca_columns)
    display(pca_df)

In [ ]:
# pca된 데이터로 optimal_K를 찾아보세요.

if PCA_mode:
    # Find optimal K
    best_pred_pca, scores_pca = find_optimal_clusters(reduced_pca, K)

In [ ]:
if quick_test:
# 클러스터 개수별 실루엣 지수를 그려주는 그래프.

    plt.figure(figsize=(8, 4))
    plt.title("Silhouette Score in range %d-%d" % (K[0], K[-1]), fontsize=14)
    plt.xlabel("Number of Clusters")
    plt.ylabel("Silhouette Score")
    plt.plot(K, scores_pca)
    plt.show()

In [ ]:
if PCA_mode:
    # PCA를 적용한 모델에 실루엣 계산
    print("Silhouette score : %.4f" % silhouette_score(reduced_pca, best_pred_pca)) # [-1, 1]

In [ ]:
if cluster_model == "hac":
    from sklearn.cluster import AgglomerativeClustering

    model = AgglomerativeClustering(n_clusters=4, affinity="euclidean", linkage="average")
    if PCA_mode:
        pred_hac = model.fit_predict(reduced_pca)
        print("Silhouette score : %.4f" % silhouette_score(reduced_pca, pred_hac)) # [-1, 1]
    else:
        pred_hac = model.fit_predict(X)
        print("Silhouette score : %.4f" % silhouette_score(X, pred_hac)) # [-1, 1]

In [ ]:
# with open(f"result_{model}_{optimal_K}_{custom_no}.txt", "w") as f:
#     f.write(str(score))
#     f.write(~~)
#     ..
#     ....